In [ ]:
import os
import shutil

def rename_and_move_wav_files(main_folder, new_folder):
    # Create the new folder if it doesn't exist
    os.makedirs(new_folder, exist_ok=True)

    for root, dirs, files in os.walk(main_folder):
        for dir in dirs:
            sub_folder = os.path.join(root, dir)
            folder_name = os.path.basename(sub_folder)
            file_names = os.listdir(sub_folder)

            for index, file_name in enumerate(file_names):
                file_path = os.path.join(sub_folder, file_name)

                if file_name.lower().endswith(".wav"):
                    new_file_name = f"{folder_name}_{index}.wav"
                    new_file_path = os.path.join(new_folder, new_file_name)
                    shutil.copy2(file_path, new_file_path)
                    print(f"Copied: {file_name}")

# Provide the main folder path and the new folder path
main_folder = "/Users/dilipharish/Master-Thesis/dh-freesound-crossmodal-search/data/audio/new_scapes"
new_folder = "/Users/dilipharish/Master-Thesis/dh-freesound-crossmodal-search/data/audio/dh-new_scapes"

# Call the rename_and_move_wav_files function
rename_and_move_wav_files(main_folder, new_folder)


In [3]:
import openai
import os
import csv
from tqdm import tqdm
import re
from decouple import config

In [13]:
# create a function that takes in a prompt and generates a soundscape description using open ai key and api
def generate_soundscape_desc(prompt_text):

        openai.api_key = config('OPENAI_API_KEY', default='')

        # Prompt the user to enter a set of tags
        # prompt = "use this table below to create a text description of the sound descriptions the first two columns are start time and end time in seconds and third column is the label of the sound and fourth is whether the sound is in the background or foreground. The natural language description should exclude the time stamps and generate 5 similar sentences describing all it's sound events. Please consider the time stamps but donot include in the description to see if the sound events are overlapping and give the description accordingly. Exclude extensive use of the word foreground and background but give just the meaning of it"
        prompt = '''use this table below to create a text description of the sound descriptions the first two columns are start /
        time and end time in seconds and third column is the label of the sound and fourth is whether the sound 
        is in the background or foreground. The natural language description should exclude the time stamps and
        generate one sentence describing all it's sound events. Please consider the time stamps but donot 
        include in the description to see if the sound events are overlapping and give the description accordingly. 
        Exclude extensive use of the word foreground and background but give just the meaning of it'''
       
        prompt += prompt_text
        # Set up the OpenAI API parameters

        model = "gpt-3.5-turbo"
        temperature = 0.1
        max_tokens = 256

        # Generate the text description using the OpenAI API
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            temperature=temperature,
            max_tokens=max_tokens,
        )

        # Print the text description
        # print(response.choices[0].text.strip())

        # return the text descriptions
        return response.choices[0].text.strip()


In [5]:
# read the folder and subfolders for .txt files and save the contents of the file in a variable and return it
def read_txt_files(folder_path):
    # read the folder and subfolders for .txt files and save the contents of the file in a variable and return it
    # create a list of all the txt files
    txt_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                txt_files.append(os.path.join(root, file))
    # print(txt_files)
    # read the contents of the txt files
    txt_files_contents = []
    for txt_file in txt_files:
        with open(txt_file, 'r') as f:
            txt_files_contents.append(f.read())
    # print(txt_files_contents)
    return txt_files_contents


In [ ]:

def read_txt_files(folder_path):
    # read the folder and subfolders for .txt files and save the contents of the file in a variable and return it
    # create a list of all the txt files
    txt_files = []
    audio_file_names = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                txt_files.append(os.path.join(root, file))

    # print(txt_files_contents)
    # return txt_files_contents

        for dir in dirs:
            sub_folder = os.path.join(root, dir)
            folder_name = os.path.basename(sub_folder)
            file_names = os.listdir(sub_folder)

            for index, file_name in enumerate(file_names):
                # file_path = os.path.join(sub_folder, file_name)

                if file_name.lower().endswith(".wav"):
                    new_file_name = f"{folder_name}_{index}.wav"
                    audio_file_names.append(new_file_name)
                    # new_file_path = os.path.join(new_folder, new_file_name)
                    # shutil.copy2(file_path, new_file_path)
                    # print(f"Copied: {file_name}")
        

    txt_files_contents = []
    for txt_file in txt_files:
        with open(txt_file, 'r') as f:
            txt_files_contents.append(f.read())
    
    return txt_files_contents, audio_file_names

In [40]:
# read the folder and subfolders for .txt files and save the contents of the file in a variable and return it
def read_txt_files_extract_audio_file_name(folder_path):
    # read the folder and subfolders for .wav files and save the contents of the file in a variable and return it
    # create a list of all the txt files
    wav_files_names = []
    for root, dirs, files in os.walk(folder_path):
        # print('files', files)
        for file in files:
            if file.endswith(".wav"):
                wav_files_names.append(os.path.join(root, file))
    # print(txt_files)
    # read the contents of the txt files
    # print('wav_files_names', wav_files_names)
    wav_filenames_af = []
    for file_name in wav_files_names:
        with open(file_name, 'r') as f:
            wav_filenames_af.append(file_name.split('/')[-1])
    # print(txt_files_contents)
    return wav_filenames_af



In [42]:
wav_file_names = read_txt_files_extract_audio_file_name('/Users/dilipharish/Master-Thesis/dh-freesound-crossmodal-search/data/audio/dh-new_scapes')

print(len(wav_file_names))

450


In [30]:
contents = read_txt_files('/Users/dilipharish/Master-Thesis/dh-freesound-crossmodal-search/data/audio/new_scapes')

print(len(contents))

450


In [14]:


# loop through each of the contents and generate a soundscape description including tqdm to show progress bar
for index, content in tqdm(enumerate(contents[:1])):
    # generated_descriptions = 'This soundscape consists of a car horn in the background, a gun shot in the foreground, as well as a drilling sound in the foreground. Additionally, these sounds overlap in certain areas. The car horn is heard before the other two sounds, and the gun shot and drilling sound follow suit. Both the gun shot and the drilling sound are heard in the foreground, while the car horn is heard in the background.'
    generated_descriptions = generate_soundscape_desc(content)
    
    for sentence in generated_descriptions.split('.'):
            #exclude the apostrophe and double quotes from the sentence
            clean_text = re.sub('[^A-Za-z\s]', '', sentence)
            clean_text = re.sub('\d+', '', clean_text)
            # sentence = sentence.replace("'", "")
            # sentence = sentence.replace('"', '')
            # writer.writerow([clean_text])
            print(clean_text)

    # print('------------------------')
    # print('Generating soundscape: {:d}/{:d}'.format(index+1, len(contents)))
    
    # #create a file name based on the index
    # file_name = 'generated_descriptions_' + str(index) + '.csv'
    # print(file_name)

    # write the generated descriptions to a csv file and if row exists append to the csv file for each sentence in the description
    # with open(file_name, 'w') as f:
    #     writer = csv.writer(f)
    #     writer.writerow(wav_file_names[index])
        

1it [00:04,  4.13s/it]

From 
 to 
 seconds a loud train whistle can be heard in the foreground followed by a background sound of chirping birds from 
 to 
 seconds



In [15]:
print('generated_descriptions', generated_descriptions)

generated_descriptions From 0.51 to 2.0 seconds a loud train whistle can be heard in the foreground, followed by a background sound of chirping birds from 2.0 to 4.0 seconds.
